# <pre><center> DATA ANALYSIS ON TITANIC CLASSIFICATION DATASET </center> <br></pre>

<pre>
Name         : data_preprocessing.ipynb
Description  : This notebook deals with the preprocessing of titanic classification dataset based on the data analysis and 
               saving the preprocessed output for training.
Language     : Python 3.6
Requirements : pandas
Input files  : ../data/train.csv, ../data/test.csv
               The dataset is downloaded from the kaggle competition 'Titanic - Machine Learning from Disaster'
               Link to the data : https://www.kaggle.com/c/titanic/data
Output files : ../preprocessed_data/train.csv, ../preprocessed_data/test.csv
</pre>

## Preprocessing is done based on the conclusion of the data analysis

* The values is column 'sex' are categorical and need to be converted to numerical for training a model.
* The missing values in column 'Age' are to be filled in both train and test data.
* The column 'Age' can be normalized since it has continuous value.
* One missing value is found in test data in 'Fare' which can be filled based on Embarked and Pclass.
* The column 'Fare' can be normalized since it has continuous value.
* The missing values in column 'Embarked' are to be filled in train data. 
* The column 'Embarked' is a categorical column and it needs to be converted to numerical column.
* The column 'Embarked' may not contain relevent information but we can try with and without this information.
* The column 'Name' can be dropped as it may not hold relevant information.
* The column 'Cabin' can be dropped as it has more than 70% missing values.

In [4]:
# Import required packages

import os
import pandas as pd

In [5]:
# Input and output paths
TRAIN_DATA_PATH = "../data/train.csv"
TEST_DATA_PATH = "../data/test.csv"
OUTPUT_PATH = "../processed_data"

In [6]:
# Create output directory
os.makedirs(OUTPUT_PATH, exist_ok=True)

## Read data from the train and test files

In [7]:
# Read the data from the file 

train_data = pd.read_csv(TRAIN_DATA_PATH, index_col="PassengerId")
test_data = pd.read_csv(TEST_DATA_PATH, index_col="PassengerId")

print(train_data.shape)
print(test_data.shape)

(891, 11)
(418, 10)


In [8]:
train_data.head()

,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
PassengerId,,,,,,,,,,,
1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [9]:
test_data.head()

,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
PassengerId,,,,,,,,,,
892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


---
## DATA PREPROCESSING

### Convert column 'Sex' to numerical

female is replaced with 0 and male is replaced with 1

In [10]:
# Numberical value to replace

sex_dict = {
    "female" : 0,
    "male" : 1
}

In [11]:
# Replace the column to numerical in train and test data
train_data_processed = train_data.replace({"Sex" : sex_dict})
test_data_processed = test_data.replace({"Sex" : sex_dict})

In [12]:
# Get the missing value count from test and train data
train_nan = train_data_processed.drop(["Survived"], axis=1).isna().sum()
test_nan = test_data_processed.isna().sum()

# Display the missing value count
nan_count = {"column" : ["Train NaN", "Test NaN"]}
for index in train_nan.index:
    nan_count[index] = [train_nan[index], test_nan[index]]

pd.DataFrame.from_dict(nan_count)

,column,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,Train NaN,0,0,0,177,0,0,0,0,687,2
1,Test NaN,0,0,0,86,0,0,0,1,327,0


## Fill Missing values in column 'Age'

The missing age values are filled with the mean of the ages with same title and sex

In [13]:
# Obtain the titles of each passenger in train data
titles = []
for item in train_data_processed["Name"]:
    title = item[item.index(",")+1:item.index(".")].strip()
    titles.append(title)
train_data_processed["Title"] = titles

# Obtain the titles of each passenger in test data
titles = []
for item in test_data_processed["Name"]:
    title = item[item.index(",")+1:item.index(".")].strip()
    titles.append(title)
test_data_processed["Title"] = titles

# Combine train and test data
combined_data = pd.concat([train_data_processed, test_data_processed])

# Obtain the mean age of passengers with same title and sex
nan_fill = {}
fill_values = []
for item in combined_data.groupby(["Title", "Sex"]):
    nan_fill[item[0]] =  item[1].Age.mean()
    fill_values.append([item[0][0], list(sex_dict.keys())[item[0][1]], item[1].Age.mean()])

pd.DataFrame(fill_values, columns=["Title", "Sex", "Average age"])

,Title,Sex,Average age
0,Capt,male,70.000000
1,Col,male,54.000000
2,Don,male,40.000000
3,Dona,female,39.000000
4,Dr,female,49.000000
5,Dr,male,42.666667
6,Jonkheer,male,38.000000
7,Lady,female,48.000000
8,Major,male,48.500000
9,Master,male,5.482642


In [14]:
# Replace the missing Age values in train data
nan_indices = train_data_processed["Age"].isna()

for index in nan_indices.index:
    if nan_indices[index]:
        train_data_processed.loc[index, "Age"] = nan_fill[(train_data_processed["Title"][index], 
                                                           train_data_processed["Sex"][index])]

In [15]:
# Replace the missing Age values in test data
nan_indices = test_data_processed["Age"].isna()

for index in nan_indices.index:
    if nan_indices[index]:
        test_data_processed.loc[index, "Age"] = nan_fill[(test_data_processed["Title"][index], 
                                                          test_data_processed["Sex"][index])]

In [16]:
# Get the missing value count from test and train data
train_nan = train_data_processed.drop(["Survived"], axis=1).isna().sum()
test_nan = test_data_processed.isna().sum()

# Display the missing value count
nan_count = {"column" : ["Train NaN", "Test NaN"]}
for index in train_nan.index:
    nan_count[index] = [train_nan[index], test_nan[index]]

pd.DataFrame.from_dict(nan_count)

,column,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Title
0,Train NaN,0,0,0,0,0,0,0,0,687,2,0
1,Test NaN,0,0,0,0,0,0,0,1,327,0,0


### Fill the missing values of 'Fare' using the mean of passengers with same Title, Sex, Pclass and Embarked

In [17]:
# Get the missinv values indices
nan_indices = test_data_processed["Fare"].isna()

combined_data = pd.concat([train_data_processed, test_data_processed])

for index in nan_indices.index:
    if nan_indices[index] == True:

        # Check if any other passenger is found with same ticket number
        ticket = test_data_processed["Ticket"][index]
        indices = combined_data[combined_data["Ticket"]==ticket].index

        # When no other passengers found with same ticket name, calculate the fare based on Title, Sex, Pclass and Embarked
        if len(indices) == 1 or combined_data["Fare"][indices].isna().sum() == len(combined_data["Fare"][indices]):
            print("Filling Fare based on Title, Sex, Pclass and Embarked")
            nan_title = test_data_processed.loc[index, "Title"]
            nan_sex = test_data_processed.loc[index, "Sex"]
            nan_pclass = test_data_processed.loc[index, "Pclass"]
            nan_embarked = test_data_processed.loc[index, "Embarked"]
            test_data_processed.loc[index, "Fare"] = \
                combined_data.groupby(["Title", "Sex", "Pclass", "Embarked"]).get_group((nan_title, nan_sex, nan_pclass, nan_embarked))["Fare"].mean()
            
        # When passengers with same ticket number is found, use the fare price from the passenger with same ticket number
        elif combined_data["Fare"][indices].isna().sum() < len(combined_data["Fare"][indices]):
            print("Filling fare based on ticket number")
            test_data_processed.loc[index, "Fare"] = combined_data["Fare"][indices].mean()


Filling Fare based on Title, Sex, Pclass and Embarked


In [18]:
# Get the missing value count from test and train data
train_nan = train_data_processed.drop(["Survived"], axis=1).isna().sum()
test_nan = test_data_processed.isna().sum()

# Display the missing value count
nan_count = {"column" : ["Train NaN", "Test NaN"]}
for index in train_nan.index:
    nan_count[index] = [train_nan[index], test_nan[index]]

pd.DataFrame.from_dict(nan_count)

,column,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Title
0,Train NaN,0,0,0,0,0,0,0,0,687,2,0
1,Test NaN,0,0,0,0,0,0,0,0,327,0,0


### Fill the missing values of 'Embarked' using the model of column 'Embarked'

In [19]:
train_data_processed["Embarked"] = train_data_processed["Embarked"].fillna(train_data_processed["Embarked"].mode()[0])

### Converting the 'Embarked' column to Numerical using one hot encoding

In [20]:
train_data_processed = pd.concat([train_data_processed, pd.get_dummies(train_data_processed.Embarked)], axis=1).drop("C", axis=1)
test_data_processed = pd.concat([test_data_processed, pd.get_dummies(test_data_processed.Embarked)], axis=1).drop("C", axis=1)

In [21]:
# Get the missing value count from test and train data
train_nan = train_data_processed.drop(["Survived"], axis=1).isna().sum()
test_nan = test_data_processed.isna().sum()

# Display the missing value count
nan_count = {"column" : ["Train NaN", "Test NaN"]}
for index in train_nan.index:
    nan_count[index] = [train_nan[index], test_nan[index]]

pd.DataFrame.from_dict(nan_count)

,column,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Title,Q,S
0,Train NaN,0,0,0,0,0,0,0,0,687,0,0,0,0
1,Test NaN,0,0,0,0,0,0,0,0,327,0,0,0,0


### Drop the columns that are not required

1. Cabin - It has lot of missing values
2. Name - It does not contain relevant information
3. Ticket - It has a lot of unique values
4. Title - It has a redundant data from Age and Sex
5. Embarked - It is not required as we have one hot encoded values

In [22]:
train_data_processed = train_data_processed.drop(["Cabin", "Name", "Ticket", "Title", "Embarked"], axis=1)
test_data_processed = test_data_processed.drop(["Cabin", "Name", "Ticket", "Title", "Embarked"], axis=1)

### Normalize the columns with continuous values - 'Age' and 'Fare'

In [23]:
# Find the mean and standard deviation of 'Age' column
combined_age = pd.concat([train_data_processed["Age"], test_data_processed["Age"]])
mean_age = combined_age.mean()
std_age = combined_age.std() 

# Normalize the train and test data with the mean and standard deviation
train_data_processed["Age"] = (train_data_processed["Age"] - mean_age) / std_age
test_data_processed["Age"] = (test_data_processed["Age"] - mean_age) / std_age

In [24]:
# Find the mean and standard deviation of 'Fare' column
combined_fare = pd.concat([train_data_processed["Fare"], test_data_processed["Fare"]])
mean_fare = combined_fare.mean()
std_fare = combined_fare.std() 

# Normalize the train and test data with the mean and standard deviation
train_data_processed["Fare"] = (train_data_processed["Fare"] - mean_fare) / std_fare
test_data_processed["Fare"] = (test_data_processed["Fare"] - mean_fare) / std_fare

### Display the missing values and info of processed data

In [25]:
# Get the missing value count from test and train data
train_nan = train_data_processed.drop(["Survived"], axis=1).isna().sum()
test_nan = test_data_processed.isna().sum()

# Display the missing value count
nan_count = {"column" : ["Train NaN", "Test NaN"]}
for index in train_nan.index:
    nan_count[index] = [train_nan[index], test_nan[index]]

pd.DataFrame.from_dict(nan_count)

,column,Pclass,Sex,Age,SibSp,Parch,Fare,Q,S
0,Train NaN,0,0,0,0,0,0,0,0
1,Test NaN,0,0,0,0,0,0,0,0


In [26]:
train_data_processed.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 891 entries, 1 to 891
Data columns (total 9 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Survived  891 non-null    int64  
 1   Pclass    891 non-null    int64  
 2   Sex       891 non-null    int64  
 3   Age       891 non-null    float64
 4   SibSp     891 non-null    int64  
 5   Parch     891 non-null    int64  
 6   Fare      891 non-null    float64
 7   Q         891 non-null    uint8  
 8   S         891 non-null    uint8  
dtypes: float64(2), int64(5), uint8(2)
memory usage: 97.4 KB


In [27]:
test_data_processed.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 418 entries, 892 to 1309
Data columns (total 8 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Pclass  418 non-null    int64  
 1   Sex     418 non-null    int64  
 2   Age     418 non-null    float64
 3   SibSp   418 non-null    int64  
 4   Parch   418 non-null    int64  
 5   Fare    418 non-null    float64
 6   Q       418 non-null    uint8  
 7   S       418 non-null    uint8  
dtypes: float64(2), int64(4), uint8(2)
memory usage: 43.7 KB


In [28]:
print(train_data_processed.shape)
print(test_data_processed.shape)

(891, 9)
(418, 8)


### Save the processed data

In [29]:
train_data_processed.to_csv(os.path.join(OUTPUT_PATH, "train.csv"))
test_data_processed.to_csv(os.path.join(OUTPUT_PATH, "test.csv"))